# Monthly Device Detections Exploration

This notebook explores monthly detection patterns across AudioMoth devices and sites. Detection counts are summarised at both the overall and device level, with additional effort-adjusted metrics used to account for variation in device availability.

## This notebook covers:

- Total detections per month (all devices combined)

- Monthly detections per device

- Effort-adjusted monthly intensity (detections per active device-day)


## Setup System Path And Get Data

In [ ]:
import sys
import os
from pathlib import Path
import pandas as pd


# Go up one level to .../audiomoth
PROJECT_ROOT = Path(os.getcwd()).resolve().parent

# Add project root to sys.path so `src` is importable
sys.path.insert(0, str(PROJECT_ROOT))

PROCESSED_DATA_PATH = Path(PROJECT_ROOT) / "data_processed" / "analysis_df.parquet"
analysis_df = pd.read_parquet(PROCESSED_DATA_PATH)

# Make pandas show more columns/rows while exploring
pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)

## Overall Detections

In [ ]:
# monthly detections (per site/device optional)
monthly_det = analysis_df.groupby(["month"]).size().rename("detections").reset_index()


# active device-days per month overall (count unique device+date pairs)
monthly_active_device_days = (
    analysis_df[["month", "device", "date"]]
    .drop_duplicates()
    .groupby(["month"])
    .size()
    .rename("active_device_days")
    .reset_index()
)

monthly_summary = monthly_det.merge(
    monthly_active_device_days, on=["month"], how="left"
)

monthly_summary["detections_per_device_day"] = (
    monthly_summary["detections"] / monthly_summary["active_device_days"]
).round(2)

monthly_summary

The following table compares monthly detection totals with the cumulative number of active device-days within each month.

Normalising monthly detections by the cumulative number of active device-days reveals substantial differences in detection intensity between months, with April showing the highest per-device-day activity.

## Save

In [ ]:
from src import data_store

data_store.save_dataframe_to_csv(
    monthly_summary,
    Path(PROJECT_ROOT) / "outputs",
    "overall_detections_monthly_summary",
)


## Detections Per Device

In [ ]:
monthly_detections_per_device = (
    analysis_df.groupby(["site", "device", "month"])
    .size()
    .rename("detections")
    .reset_index()
    .sort_values(["site", "device", "month"])
    .reset_index(drop=True)
)

# active device-days per month for each device (count unique device+date pairs)
monthly_active_device_days = (
    analysis_df[["device", "month", "date"]]
    .drop_duplicates()
    .groupby(["device", "month"])
    .size()
    .rename("active_device_days")
    .reset_index()
)

monthly_detections_per_device = monthly_detections_per_device.merge(
    monthly_active_device_days, on=["device", "month"], how="left"
)
monthly_detections_per_device["detections_per_device_day"] = (
    monthly_detections_per_device["detections"]
    / monthly_detections_per_device["active_device_days"]
).round(2)

monthly_detections_per_device

### Save

In [ ]:
data_store.save_dataframe_to_csv(
    monthly_detections_per_device,
    Path(PROJECT_ROOT) / "outputs",
    "device_detections_monthly_summary",
)

When adjusted for daily recording effort, detection intensity increases markedly from February to March across most devices, indicating a strong seasonal signal. April shows particularly high per-device-day activity at several sites, although these values are based on fewer active days and may reflect concentrated peak activity periods. Device-level differences persist within sites, suggesting that local habitat characteristics or recorder placement influence acoustic intensity.

### Key takeaways:

- Seasonal increase from February → March.

- April shows strong intensification at certain sites.

- Device-level variation persists even after effort correction.

- Helman Tor consistently lower intensity.

- Higher Trevilmick and Lowertown show particularly high spring intensity.